<a href="https://colab.research.google.com/github/weso500/QMLPublicationRuns/blob/main/FinalRunsDataPrep_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyreadr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 7.3 MB/s eta 0:00:00


In [2]:
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, mean_squared_error
from matplotlib import pyplot
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import zipfile
import pandas as pd

In [3]:
# Path to your ZIP file
zip_path = '/content/drive/MyDrive/FDA Journal/University of Ottawa Rolling-element Dataset – Vibration and Acoustic Faults under Constant Load and Speed conditions (UORED-VAFCLS).zip'

# Create a dictionary to store DataFrames
dataframes = {}

# Open the ZIP file
with zipfile.ZipFile(zip_path, 'r') as z:
    # Loop through each file in the ZIP
    for filename in z.namelist():
        if filename.endswith('.csv'):
            # Read each CSV into a DataFrame
            with z.open(filename) as f:
                df = pd.read_csv(f)
                dataframes[filename] = df

# Create an empty list to store healthy dataframes
healthy_dfs = []

# Iterate through the dataframes dictionary
for filename, df in dataframes.items():
    # Check if 'Healthy' is in the filename
    if 'Healthy' in filename:
        healthy_dfs.append(df)

# Concatenate the healthy dataframes into a single dataframe
healthy_merged_df = pd.concat(healthy_dfs, ignore_index=True)


# Drop the 'Speed' and 'Load' columns
healthy_merged_df = healthy_merged_df.drop(['Speed', 'Load'], axis=1)

# Display the first few rows of the modified dataframe
display(healthy_merged_df.head())


,Accelerometer,Acoustic,Temperature Difference
0,4.454746,-0.004864,3.760168
1,-0.638000,-0.004206,3.434396
2,-1.443815,-0.003877,4.255864
3,-2.797583,-0.003549,3.435602
4,-3.281071,-0.002562,3.107306


In [4]:
def create_windows(df, window_size, overlap):
    windows = []
    step_size = window_size
    for i in range(0, len(df) - window_size + 1, step_size):
        window = df.iloc[i:i + window_size]
        windows.append(window)
    return windows

window_size = 4200
overlap = int(window_size * 0.5)

healthy_windows = create_windows(healthy_merged_df, window_size, overlap)

# Create empty dictionaries to store data for each sensor
sensor_data = {
    'Accelerometer': [],
    'Acoustic': [],
    'Temperature Difference': []
}

# Iterate through each window
for window in healthy_windows:
    # Extract data for each sensor and append to the respective list
    sensor_data['Accelerometer'].append(window['Accelerometer'].values)
    sensor_data['Acoustic'].append(window['Acoustic'].values)
    sensor_data['Temperature Difference'].append(window['Temperature Difference'].values)

# Convert the lists of arrays into dataframes
accelerometer_df = pd.DataFrame(sensor_data['Accelerometer'])
acoustic_df = pd.DataFrame(sensor_data['Acoustic'])
temperature_df = pd.DataFrame(sensor_data['Temperature Difference'])

# Display the shapes of the new dataframes to confirm
print(f"Shape of Accelerometer dataframe: {accelerometer_df.shape}")
print(f"Shape of Acoustic dataframe: {acoustic_df.shape}")
print(f"Shape of Temperature Difference dataframe: {temperature_df.shape}")

Shape of Accelerometer dataframe: (2000, 4200)
Shape of Acoustic dataframe: (2000, 4200)
Shape of Temperature Difference dataframe: (2000, 4200)


In [5]:
# Path to your ZIP file
zip_path = '/content/drive/MyDrive/FDA Journal/University of Ottawa Rolling-element Dataset – Vibration and Acoustic Faults under Constant Load and Speed conditions (UORED-VAFCLS).zip'

# Create a dictionary to store DataFrames
fault_dataframes = {}

# Open the ZIP file
with zipfile.ZipFile(zip_path, 'r') as z:
    # Loop through each file in the ZIP
    for filename in z.namelist():
        if filename.endswith('.csv') and 'Fault' in filename:
            # Read each CSV into a DataFrame
            with z.open(filename) as f:
                df = pd.read_csv(f)
                fault_dataframes[filename] = df

# Create empty lists to store dataframes for developing and faulty conditions
fault_developing_dfs = []
faulty_dfs = []

# Iterate through the fault_dataframes dictionary
for filename, df in fault_dataframes.items():
    # Extract the fault condition from the filename using the labeling scheme
    # Assuming the format is Type_Bearing_Condition.csv (e.g., I_1_1.csv)
    parts = filename.split('/')[-1].split('.')[0].split('_')
    if len(parts) >= 3:
        condition = parts[2]
        if condition == '1':
            fault_developing_dfs.append(df)
        elif condition == '2':
            faulty_dfs.append(df)

# Concatenate the dataframes for each condition
fault_developing_merged_df = pd.concat(fault_developing_dfs, ignore_index=True)
faulty_merged_df = pd.concat(faulty_dfs, ignore_index=True)

# Display the first few rows of the merged dataframes
print("Fault Developing Merged DataFrame:")
display(fault_developing_merged_df.head())

print("\nFaulty Merged DataFrame:")
display(faulty_merged_df.head())


# Drop the 'Speed' and 'Load' columns from fault dataframes
fault_developing_merged_df = fault_developing_merged_df.drop(['Speed', 'Load'], axis=1)
faulty_merged_df = faulty_merged_df.drop(['Speed', 'Load'], axis=1)

# Display the first few rows of the modified fault dataframes
print("Fault Developing Merged DataFrame after normalization and dropping columns:")
display(fault_developing_merged_df.head())

print("\nFaulty Merged DataFrame after normalization and dropping columns:")
display(faulty_merged_df.head())


# Apply windowing to the fault developing and faulty dataframes
fault_developing_windows = create_windows(fault_developing_merged_df, window_size, overlap)
faulty_windows = create_windows(faulty_merged_df, window_size, overlap)

# Create empty dictionaries to store data for each sensor
sensor_data_faulty = {
    'Accelerometer': [],
    'Acoustic': [],
    'Temperature Difference': []
}

# Iterate through each window
for window in fault_developing_windows:
    # Extract data for each sensor and append to the respective list
    sensor_data_faulty['Accelerometer'].append(window['Accelerometer'].values)
    sensor_data_faulty['Acoustic'].append(window['Acoustic'].values)
    sensor_data_faulty['Temperature Difference'].append(window['Temperature Difference'].values)

# Convert the lists of arrays into dataframes
accelerometer_df_faulty = pd.DataFrame(sensor_data_faulty['Accelerometer'])
acoustic_df_faulty = pd.DataFrame(sensor_data_faulty['Acoustic'])
temperature_df_faulty = pd.DataFrame(sensor_data_faulty['Temperature Difference'])

# Display the shapes of the new dataframes to confirm
print(f"Shape of Accelerometer dataframe: {accelerometer_df_faulty.shape}")
print(f"Shape of Acoustic dataframe: {acoustic_df_faulty.shape}")
print(f"Shape of Temperature Difference dataframe: {temperature_df_faulty.shape}")

Fault Developing Merged DataFrame:


,Accelerometer,Acoustic,Speed,Load,Temperature Difference
0,19.700756,0.074370,1819,400,22.575598
1,19.829687,0.083576,0,0,21.964033
2,22.021502,0.088179,0,0,22.583118
3,23.923225,0.088508,0,0,21.308137
4,24.148853,0.092453,0,0,22.106005



Faulty Merged DataFrame:


,Accelerometer,Acoustic,Speed,Load,Temperature Difference
0,34.785607,-0.025905,1819,400,24.261550
1,35.591422,-0.024261,0,0,24.221602
2,37.815471,-0.019987,0,0,25.064400
3,36.784028,-0.018672,0,0,24.425321
4,40.942032,-0.021302,0,0,26.213233


Fault Developing Merged DataFrame after normalization and dropping columns:


,Accelerometer,Acoustic,Temperature Difference
0,19.700756,0.074370,22.575598
1,19.829687,0.083576,21.964033
2,22.021502,0.088179,22.583118
3,23.923225,0.088508,21.308137
4,24.148853,0.092453,22.106005



Faulty Merged DataFrame after normalization and dropping columns:


,Accelerometer,Acoustic,Temperature Difference
0,34.785607,-0.025905,24.261550
1,35.591422,-0.024261,24.221602
2,37.815471,-0.019987,25.064400
3,36.784028,-0.018672,24.425321
4,40.942032,-0.021302,26.213233


Shape of Accelerometer dataframe: (2000, 4200)
Shape of Acoustic dataframe: (2000, 4200)
Shape of Temperature Difference dataframe: (2000, 4200)


In [6]:
def calculate_multimodal_features_qkernel(
    vib_windows,
    acous_windows,
    temp_windows,
    fs=42000,
    n_bands=8,
    nperseg=1024,
    max_lag_sec=0.01,
    scale=True,
    return_angles=False
):
    """
    Returns DataFrame with 11 features optimized for quantum kernel SVM:
    - 6 engineered features (coherence, lag correlation, RMS)
    - 5 raw/FFT features (Acous_Entropy + top FFT bins from vib/acous)
    """
    import numpy as np
    import pandas as pd
    import scipy.signal as ss
    from scipy.stats import kurtosis
    from scipy.fft import rfft
    from sklearn.preprocessing import StandardScaler

    n_windows, window_len = vib_windows.shape
    max_lag = int(max_lag_sec * fs)
    nyq = fs / 2.0

    # Band edges for coherence
    min_resolvable = float(fs) / float(nperseg)
    min_freq = max(min_resolvable, 1.0)
    band_edges = np.logspace(np.log10(min_freq), np.log10(nyq), n_bands + 1)

    rows = []

    for w in range(n_windows):
        vib = vib_windows[w].astype(float)
        ac  = acous_windows[w].astype(float)
        tmp = temp_windows[w].astype(float)

        # --- Engineered features ---
        vib_rms = np.sqrt(np.mean(vib**2))

        xi = vib - vib.mean()
        xj = tmp - tmp.mean()
        corr = ss.correlate(xi, xj, mode='full')
        lags = np.arange(-len(xi)+1, len(xi))
        mask = (lags >= -max_lag) & (lags <= max_lag)
        corr_mask = corr[mask] if mask.size else np.array([])
        lag_peak = float(corr_mask[np.argmax(np.abs(corr_mask))] / (np.linalg.norm(xi)*np.linalg.norm(xj)+1e-12)) if corr_mask.size else 0.0

        def band_coherence(x, y):
            f_coh, Cxy = ss.coherence(x, y, fs=fs, nperseg=nperseg)
            vals = []
            for b in range(n_bands):
                low, high = band_edges[b], band_edges[b+1]
                idx = np.where((f_coh >= low) & (f_coh < high))[0]
                vals.append(np.mean(Cxy[idx]) if idx.size else 0.0)
            return np.array(vals)

        vib_ac_coh  = band_coherence(vib, ac)
        vib_tmp_coh = band_coherence(vib, tmp)
        ac_tmp_coh  = band_coherence(ac, tmp)

        def safe_get(arr, idx, default=0.0):
            return float(arr[idx]) if (hasattr(arr, "__len__") and idx < len(arr)) else default

        coh_vac_b1 = safe_get(vib_ac_coh, 0)
        coh_vac_b2 = safe_get(vib_ac_coh, 1)
        coh_vt_b2  = safe_get(vib_tmp_coh, 1)
        coh_at_b2  = safe_get(ac_tmp_coh, 1)

        # --- Compressed raw / FFT features ---
        yf_vib = np.abs(rfft(vib))
        yf_ac  = np.abs(rfft(ac))

        top_fft_vib = np.sort(yf_vib)[-2:]   # 2 largest vibration FFT bins
        top_fft_ac  = np.sort(yf_ac)[-2:]    # 2 largest acoustic FFT bins

        # Acoustic spectral entropy
        p = yf_ac**2
        total = p.sum()
        if total > 0:
            p_norm = p / total
            p_norm[p_norm == 0] = 1e-12
            ac_entropy = -np.sum(p_norm * np.log2(p_norm))
        else:
            ac_entropy = 0.0

        # Combine features
        features = [
            coh_vac_b1,
            coh_vac_b2,
            coh_vt_b2,
            coh_at_b2,
            lag_peak,
            vib_rms,
            ac_entropy,
            top_fft_vib[0],
            top_fft_vib[1],
            top_fft_ac[0],
            top_fft_ac[1]
        ]
        rows.append(features)

    cols = [
        'Coh_Vib_Acous_B1', 'Coh_Vib_Acous_B2',
        'Coh_Vib_Temp_B2', 'Coh_Acous_Temp_B2',
        'LagCorr_Vib_Temp_Peak', 'Vib_RMS',
        'Acous_Entropy',
        'Vib_FFT1', 'Vib_FFT2',
        'Acous_FFT1', 'Acous_FFT2'
    ]

    df = pd.DataFrame(rows, columns=cols)

    if scale:
        scaler = StandardScaler()
        df.loc[:, :] = scaler.fit_transform(df.values)

    if return_angles:
        df.loc[:, :] = np.tanh(df.values) * np.pi

    return df





In [7]:
# convert lists-of-windows to stacked arrays if needed
vib = np.vstack(sensor_data['Accelerometer'])
ac  = np.vstack(sensor_data['Acoustic'])
temp = np.vstack(sensor_data['Temperature Difference'])
fault_vib = np.vstack(sensor_data_faulty['Accelerometer'])
fault_ac = np.vstack(sensor_data_faulty['Acoustic'])
fault_temp = np.vstack(sensor_data_faulty['Temperature Difference'])


# Take 500 samples from each dataframe for the test sets
test_vib = vib[:500]
test_ac = ac[:500]
test_temp = temp[:500]

# Keep the remaining data for training
train_vib = vib[500:]
train_ac = ac[500:]
train_temp = temp[500:]

# Take 10 samples from faulty dataframes and append to test sets
test_vib = np.vstack([test_vib, fault_vib[:100]])
test_ac = np.vstack([test_ac, fault_ac[:100]])
test_temp = np.vstack([test_temp, fault_temp[:100]])

# extract features for the training data
df = calculate_multimodal_features_qkernel(
    vib_windows = train_vib,
    acous_windows = train_ac,
    temp_windows = train_temp,
    fs = 42000,
    n_bands = 8,
    nperseg = 1024,
    max_lag_sec = 0.01,
    scale = False,
    return_angles = False
)

df_test = calculate_multimodal_features_qkernel(
    vib_windows = test_vib,
    acous_windows = test_ac,
    temp_windows = test_temp,
    fs = 42000,
    n_bands = 8,
    nperseg = 1024,
    max_lag_sec = 0.01,
    scale = False,
    return_angles = False
)

display(df.head())
display(df_test.head())
print(df.describe().T)   # quick sanity: check non-zero coherence stats
print(df.shape)

,Coh_Vib_Acous_B1,Coh_Vib_Acous_B2,Coh_Vib_Temp_B2,Coh_Acous_Temp_B2,LagCorr_Vib_Temp_Peak,Vib_RMS,Acous_Entropy,Vib_FFT1,Vib_FFT2,Acous_FFT1,Acous_FFT2
0,0.134229,0.261817,0.270336,0.033065,0.034477,32.793376,5.392567,17184.082443,19084.495519,22.405127,25.811047
1,0.256095,0.343843,0.032532,0.061574,0.039508,31.954009,5.234968,14780.414766,15914.607301,30.155074,30.362133
2,0.120565,0.240023,0.158226,0.137878,0.037644,31.851533,4.897254,15337.711258,18801.344293,27.653095,42.468650
3,0.282216,0.059204,0.312438,0.153320,-0.034408,33.148847,4.607874,18592.740129,21787.230654,36.129959,49.393887
4,0.156204,0.281034,0.334055,0.176941,0.037699,33.765916,4.642709,17864.391652,18446.825859,33.021994,52.002163


,Coh_Vib_Acous_B1,Coh_Vib_Acous_B2,Coh_Vib_Temp_B2,Coh_Acous_Temp_B2,LagCorr_Vib_Temp_Peak,Vib_RMS,Acous_Entropy,Vib_FFT1,Vib_FFT2,Acous_FFT1,Acous_FFT2
0,0.137663,0.392774,0.272623,0.036901,0.051723,3.150627,2.440444,1661.860704,5055.604331,7.276330,18.266779
1,0.266726,0.696038,0.254271,0.288340,-0.058176,3.362751,4.062654,2162.745602,5223.245991,5.467823,7.436546
2,0.108777,0.405616,0.425667,0.173773,0.049464,3.253601,3.897669,1748.742637,5205.292415,5.326317,8.768574
3,0.211598,0.322585,0.187016,0.279666,-0.055470,3.080352,3.774093,2139.947676,5281.877022,5.253129,8.245843
4,0.317516,0.251304,0.118870,0.153708,-0.040358,2.961580,2.149421,1624.076309,5318.557596,5.222330,18.081058


                        count           mean            std          min  \
Coh_Vib_Acous_B1       1500.0       0.196455       0.149098     0.002739   
Coh_Vib_Acous_B2       1500.0       0.268568       0.269705     0.002097   
Coh_Vib_Temp_B2        1500.0       0.137117       0.103349     0.001113   
Coh_Acous_Temp_B2      1500.0       0.146024       0.098831     0.001325   
LagCorr_Vib_Temp_Peak  1500.0      -0.001673       0.044116    -0.073212   
Vib_RMS                1500.0      30.003996      48.485327     2.642546   
Acous_Entropy          1500.0       3.087244       1.464513     0.000038   
Vib_FFT1               1500.0    4552.006013    4046.700077  1609.212901   
Vib_FFT2               1500.0  108720.512249  209609.742873  1900.889133   
Acous_FFT1             1500.0      24.813498      11.304434     4.320726   
Acous_FFT2             1500.0     630.425327    1810.720315     5.821857   

                               25%          50%           75%            max  
Coh_Vib_

In [8]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data (merged_features_df) and transform it
train_scaled = scaler.fit_transform(df)

# Transform the test data (test_df) using the scaler fitted on the training data
test_df_scaled = scaler.transform(df_test)


In [9]:
# Save the pca_featurestest_df as a CSV file
train_scaled_df = pd.DataFrame(train_scaled, columns=df.columns)
train_scaled_df.to_csv('Train_11_final.csv', index=False)

print("pca_featurestest_df saved to 'pca_test_features_with_target.csv'")

pca_featurestest_df saved to 'pca_test_features_with_target.csv'


In [10]:
# Save the pca_features_df as a CSV file
test_df_scaled_df = pd.DataFrame(test_df_scaled, columns=df_test.columns)
test_df_scaled_df.to_csv('Test_11_final.csv', index=False)

print("pca_features_df saved to 'pca_train_features.csv'")

pca_features_df saved to 'pca_train_features.csv'
